In [1]:
#import pydicom
import os
import numpy as np
import pandas as pd
import cv2
from tqdm.notebook import tqdm

#한글로 이미지 읽기 쓰기

def han_imread(ld_dir, color_mode = True) :
    img_array = np.fromfile(ld_dir, np.uint8)
    if color_mode == True :
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    else :
        img = cv2.imdecode(img_array, cv2.IMREAD_GRAYSCALE)
    return img

def han_imwrite(sv_dir, image) :
    new_img_name = sv_dir
    extension = os.path.splitext(new_img_name)[1] # 이미지 확장자
    result, encoded_img = cv2.imencode(extension, image)
    if result:
        with open(new_img_name, mode='w+b') as f:
            encoded_img.tofile(f)

root_dir = 'data/train/' # 이미지 파일이 있는 디렉토리
os.makedirs('Data/Training_whole/PNG', exist_ok=True)
os.makedirs('Data/Training_whole/PNG_384', exist_ok=True) # 사이즈는 훈련할때 미리 원하는 사이즈로 있어야 빠름. 원하는 preprocessing이나 사이즈는 미리 넣어주세요
meta_dict_list = []
i = 0
for (root, dirs, files) in tqdm(os.walk(root_dir)): # 디렉토리 내의 모든 파일 순회
    for file_name in tqdm(files):
        if ('png' in file_name) or ('jpg' in file_name) or ('bmp' in file_name) : # 무슨 확장자가 있는지 몰라서 일단 썼습니다
            
            meta_dict = dict()
            png_img_name = os.path.basename(root) + '_' + file_name.split('.')[0] + '.png'
            meta_dict['id'] = i
            meta_dict['filename'] = png_img_name
            
            meta_dict['label'] = os.path.basename(root) # basename : 파일 이름만 출력한다
            meta_dict_list.append(meta_dict)
            
            img = han_imread(os.path.join(root, file_name))
            img_224 = cv2.resize(img, dsize = (384,384), interpolation = cv2.INTER_AREA)

            han_imwrite( os.path.join('Data/Training_whole/PNG', png_img_name), img)
            han_imwrite( os.path.join('Data/Training_whole/PNG_384', png_img_name), img_224)
            i += 1

meta_train = pd.DataFrame(meta_dict_list)
meta_train.to_csv('train.csv', index = False)


In [4]:
meta_tarin_k = pd.read_csv('./train.csv')

In [5]:
meta_tarin_k

,id,filename,label
0,0,울음_12.png,울음
1,1,울음_11.png,울음
2,2,울음_16.png,울음
3,3,울음_17.png,울음
4,4,울음_21.png,울음
...,...,...,...
3452,3452,걸레받이수정_160.png,걸레받이수정
3453,3453,걸레받이수정_214.png,걸레받이수정
3454,3454,걸레받이수정_284.png,걸레받이수정
3455,3455,걸레받이수정_198.png,걸레받이수정
